In [1]:
%load_ext tensorboard
import pandas as pd
import torch
from torch import nn
from transformers import DistilBertTokenizer, DistilBertModel
from torch.utils.data import DataLoader,Dataset
from matplotlib import pyplot as plt
from metric_writer.src import MetricWriter
from metric_writer.src.configs.defaults import SimpleClf
# Define script variables
BATCH_SIZE   = 32
N_CLASSES    = 2
INP_SIZE     = 768
EPOCHS       = 100
STEP         = 5
TRAIN_SPLIT  = .9

In [3]:
def saveTensors():
    """
        Save the text as tensors so we don't need to do this everytime.
        
        Apply the "reduce" function to get the representation for the outut space
        
        NOTE: when doing this you should always prevent the gradient from accumilating i.e. using "torch.no_grad()"
    """
    from utils import reductions
    from tqdm import tqdm
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    model=DistilBertModel.from_pretrained('distilbert-base-uncased').eval()
    df = pd.read_csv("data/train.csv")
    def outp_vec(dim):
        v = [0,0]
        v[dim]=1.0
        return v
    with torch.no_grad():torch.save( [ 
                    (
                        reductions.mean( model(**tokenizer( df.text.values[idx] , return_tensors="pt")) ),
                        int(df.target.values[idx])
                    )  for idx in tqdm(range(len(df.text.values[:500]))) 
                ], 
            'data/train.data' 
    )
    
saveTensors()

In [5]:
class TweetDataSet(Dataset):
    
    def __init__(self):
        super( TweetDataSet, self ).__init__()
        self.data = torch.load('data/train.data')     
        
    def __getitem__(self,index):
        return self.data[index]
    def __len__(self):return len( self.data )

In [7]:
# Create Variables
dataset     = TweetDataSet()
train_data  = DataLoader(dataset[ : int(len(dataset)*TRAIN_SPLIT) ] ,batch_size=BATCH_SIZE )
test_data   = DataLoader(dataset[ int(len(dataset)*TRAIN_SPLIT) : ] ,batch_size=int(len(dataset)*TRAIN_SPLIT) ) # we aren't using the gradient, and there's so little data we can just do one batch
model       = TweetClf()
optim       = torch.optim.AdamW( model.parameters() )
loss_fn     = nn.CrossEntropyLoss()
mw          = MetricWriter( SimpleClf() )

FileNotFoundError: [Errno 2] No such file or directory: 'data/train.data'

In [ ]:
def test():
    model.eval()
    x,y = list(test_data)
    with torch.no_grad():y_hat = model( x )
    loss = loss_fn( y_hat,y )
    acc_y = y_hat.argmax(dim=-1)
    acc = round(100*( ( sum( 1 if acc_y[i] == labels[i] else 0 for i in range(len(acc_y)) ) )/len(test_data) ))
    print( f'Test Accuracy: {acc}% ')
    print( f'Test Loss: {liss.item()}% ')

In [81]:
def train():
    from tqdm import tqdm
    i = 0
    for e in range(EPOCHS):
        model.train()
        for x,y in tqdm(train_ata):
            y_hat = model(x)
            loss = loss_fn( y_hat,y ).backward()
            optim.step()
            if i%STEP == 0:mw( y_hat,y,loss_fn )
            i+=1
        test()

In [82]:
%tensorboard --logdir=runs
train()

Reusing TensorBoard on port 6006 (pid 13037), started 3:13:02 ago. (Use '!kill 13037' to kill it.)



  0%|          | 0/16 [00:00<?, ?it/s]

 50%|█████     | 8/16 [00:00<00:00, 75.49it/s]

 81%|████████▏ | 13/16 [00:00<00:00, 64.91it/s]

100%|██████████| 16/16 [00:00<00:00, 57.63it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

 31%|███▏      | 5/16 [00:00<00:00, 45.88it/s]

 56%|█████▋    | 9/16 [00:00<00:00, 41.95it/s]

 81%|████████▏ | 13/16 [00:00<00:00, 39.23it/s]

100%|██████████| 16/16 [00:00<00:00, 38.25it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

 31%|███▏      | 5/16 [00:00<00:00, 42.90it/s]

 69%|██████▉   | 11/16 [00:00<00:00, 45.81it/s]

100%|██████████| 16/16 [00:00<00:00, 51.41it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

 38%|███▊      | 6/16 [00:00<00:00, 54.36it/s]

 69%|██████▉   | 11/16 [00:00<00:00, 47.75it/s]

 94%|█████████▍| 15/16 [00:00<00:00, 44.29it/s]

100%|██████████| 16/16 [00:00<00:00, 43.49it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

 25%|██▌       | 4/16 [00:00<00:00, 36.46it/s]

 38%|███▊      | 6/16 [00:00<00:00, 27.21it/s]

 50%|█████     | 8/16

  0%|          | 0/16 [00:00<?, ?it/s]

 31%|███▏      | 5/16 [00:00<00:00, 43.06it/s]

 62%|██████▎   | 10/16 [00:00<00:00, 43.99it/s]

 94%|█████████▍| 15/16 [00:00<00:00, 45.63it/s]

100%|██████████| 16/16 [00:00<00:00, 46.34it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

 31%|███▏      | 5/16 [00:00<00:00, 44.06it/s]

 62%|██████▎   | 10/16 [00:00<00:00, 45.67it/s]

 94%|█████████▍| 15/16 [00:00<00:00, 45.85it/s]

100%|██████████| 16/16 [00:00<00:00, 46.44it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

 25%|██▌       | 4/16 [00:00<00:00, 37.33it/s]

 56%|█████▋    | 9/16 [00:00<00:00, 40.09it/s]

 88%|████████▊ | 14/16 [00:00<00:00, 41.66it/s]

100%|██████████| 16/16 [00:00<00:00, 43.94it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

 31%|███▏      | 5/16 [00:00<00:00, 46.18it/s]

 62%|██████▎   | 10/16 [00:00<00:00, 46.31it/s]

 94%|█████████▍| 15/16 [00:00<00:00, 46.39it/s]

100%|██████████| 16/16 [00:00<00:00, 46.92it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

 25%|██▌       | 4/16

 69%|██████▉   | 11/16 [00:00<00:00, 47.57it/s]

100%|██████████| 16/16 [00:00<00:00, 49.80it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

 38%|███▊      | 6/16 [00:00<00:00, 54.31it/s]

 69%|██████▉   | 11/16 [00:00<00:00, 52.93it/s]

100%|██████████| 16/16 [00:00<00:00, 52.15it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

 38%|███▊      | 6/16 [00:00<00:00, 51.32it/s]

 75%|███████▌  | 12/16 [00:00<00:00, 51.72it/s]

100%|██████████| 16/16 [00:00<00:00, 51.91it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

 38%|███▊      | 6/16 [00:00<00:00, 52.11it/s]

 75%|███████▌  | 12/16 [00:00<00:00, 51.96it/s]

100%|██████████| 16/16 [00:00<00:00, 50.38it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

 38%|███▊      | 6/16 [00:00<00:00, 53.00it/s]

 69%|██████▉   | 11/16 [00:00<00:00, 51.66it/s]

100%|██████████| 16/16 [00:00<00:00, 50.23it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

 31%|███▏      | 5/16 [00:00<00:00, 41.36it/s]

 62%|██████▎   | 10/16 [00:00<00:00, 43.29it/s]

100%|██████████| 16/1